# Encoder-decoder Architectures

The following notebook provides development of two encoder-decoder CNN architectures (U-Net and a simple 4-layer architecture). Both models are trained useing the ISPRS dataset of the city of Vaihingen in Germany: (http://www2.isprs.org/commissions/comm3/wg4/2d-sem-label-vaihingen.html).


Code for the UNet modle influenced by the following blog:
https://www.kaggle.com/toregil/a-lung-u-net-in-keras

Code for 4-layer encoder-decoder modle influenced by the following blog:
https://blog.keras.io/building-autoencoders-in-keras.html

Code for the UNet modle influenced by the following blog:
https://www.kaggle.com/toregil/a-lung-u-net-in-keras

Code for 4-layer encoder-decoder modle influenced by the following blog:
https://blog.keras.io/building-autoencoders-in-keras.html

 Import libraries

In [7]:

import keras
from keras.models import Model,Sequential
from keras.callbacks import ModelCheckpoint,TensorBoard
from keras.layers import Input, Dense, Conv2D, MaxPool2D, UpSampling2D, Dropout, Activation,concatenate,MaxPooling2D
from keras.optimizers import Adam

import os
import numpy as np
import random
import cv2
import re
import pickle
from sklearn.metrics import precision_score, f1_score,classification_report,recall_score,precision_recall_fscore_support

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from IPython.core.display import Image, display

Define global variables:

In [8]:
IMG_PATH = '/home/niamh/notebooks/PATCH_BASED/patch_32_stride_18/train_patches/total_train/'
GT_PATH = '/home/niamh/notebooks/AUTOENCODER/GT_patches_32_stride18/train_patches/total_train/'

# Dimensions of input images for both models
IMG_HEIGHT, IMG_WIDTH, CHANNELS = 32, 32, 3

# For useing a sample of the trainnig data
SAMPLE_RATE = 1


# 1. Data Pre-processing

In [26]:
all_images = [x for x in sorted(os.listdir(IMG_PATH)) if x[-4:] == '.png']
all_images = random.sample(all_images, int(len(all_images)*SAMPLE_RATE))

# Initialise array for training  images
x_data = np.empty((len(all_images), IMG_HEIGHT, IMG_WIDTH, CHANNELS), dtype='float32')

# Initialise array for ground truth of training images
y_data = np.empty((len(all_images), IMG_HEIGHT, IMG_WIDTH, CHANNELS), dtype='float32')

# Number of training examples
train_count = len(all_images)
print ('Number of training examples: ',train_count)

# Itterate through each image resize, normalize and add to training array 
for i, name in enumerate(all_images):
    im = cv2.imread(IMG_PATH + name, cv2.IMREAD_UNCHANGED).astype("int16").astype('float32')/255.
    im = cv2.resize(im, dsize=(IMG_WIDTH, IMG_HEIGHT), interpolation=cv2.INTER_LANCZOS4)
    #im = (im - np.min(im)) / (np.max(im) - np.min(im))
    x_data[i] = im
    
    row = str(re.search('row(\d+)', name).group())
    col = str(re.search('col(\d+)', name).group())
    gt = cv2.imread(GT_PATH +name[0:8]+row+'_'+col+'_.png', cv2.IMREAD_UNCHANGED).astype('float32')/255.
    gt = cv2.resize(gt, dsize=(IMG_WIDTH, IMG_HEIGHT), interpolation=cv2.INTER_NEAREST)
    
    y_data[i] = gt
    
    if i%1000 == 0: print('Processed {} of {}'.format(i, train_count))

#### Display some examples from the dataset:

In [27]:
n = 5
plt.figure(figsize=(20, 10)) 

for i in range(n):
    i = i+1
    idx = random.randint(0, len(x_data))
    
    # display original
    ax = plt.subplot(2, n, i)
    plt.imshow(x_data[idx].reshape(IMG_HEIGHT, IMG_WIDTH,3))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    plt.title('Image example '+ str(idx))
    
    # display reconstruction
    ax = plt.subplot(2, n, i + n)
    plt.imshow(y_data[idx].reshape(IMG_HEIGHT, IMG_WIDTH, 3))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    plt.title('Ground truth example '+ str(idx))

    
plt.show()

# 2. 4-Layer Encoder-decoder Model

Parameters for model and training:

In [11]:
# Model Params
optimizer='adadelta'
loss='mean_squared_error'

# Training Params
best_weights_filepath = '4layer_encoder_decoder_best_weights.hdf5'
epochs = 10
batch_size = 10
validation_split=0.3

Build model architecture:

In [28]:
input_img = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))

# Encoder
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# Decoder
x = Conv2D(32, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

encoder_decoder = Model(input_img, decoded)
encoder_decoder.compile(optimizer='adadelta', loss='mean_squared_error', metrics=['accuracy'])
encoder_decoder.summary()

Compile 4-layer encoder-decoder model:

In [13]:
encoder_decoder.compile(optimizer=optimizer, loss=loss,metrics=['accuracy'])

Train 4-layer encoder-decoder model:

In [29]:
# Save best weights of model dependent on the lowest validation loss to load for testing
mcp = ModelCheckpoint(best_weights_filepath, monitor="val_loss",
                      save_best_only=True, save_weights_only=False)

# Train model
history =encoder_decoder.fit(x_data, y_data,
                epochs=epochs,
                batch_size=batch_size,
                shuffle=True,
                validation_split=validation_split,
                        callbacks = [mcp])

Display and record training info:

In [30]:
loss = history.history['loss']
val_loss = history.history['val_loss']

acc = history.history['acc']
val_acc = history.history['val_acc']

loss_fig = plt.figure()
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.plot(loss, 'blue', label='Training Loss')
plt.plot(val_loss, 'green', label='Validation Loss')
plt.xticks(range(0,epochs)[0::2])
plt.legend()
plt.show()
loss_fig.savefig('4layer_encoder_decoder_loss.png')


acc_fig = plt.figure()
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.plot(acc, 'blue', label='Training Accuracy')
plt.plot(val_acc, 'green', label='Validation Accuracy')
plt.xticks(range(0,epochs)[0::2])
plt.legend()
plt.show()
acc_fig.savefig('4layer_encoder_decoder_accuracy.png')


Load best weights and save final model:

In [16]:
# Load best weights form training
encoder_decoder.load_weights(best_weights_filepath)

# Save final model
filepath = "4layer_encoder_decoder.mod"
encoder_decoder.save(filepath)

# 3. U-Net Model

Parameters for model and training:

In [17]:
# Model Params
optimizer=Adam(2e-4)
loss='mean_squared_error'

# Training Params
best_weights_filepath = 'UNet_best_weights.hdf5'
epochs = 10
batch_size = 10
validation_split=0.3

Build U-Net model architecture:

In [31]:
input_layer = Input(shape=x_data.shape[1:])

c1 = Conv2D(filters=8, kernel_size=(3,3), activation='relu', padding='same')(input_layer)
l = MaxPool2D(strides=(2,2))(c1)

c2 = Conv2D(filters=16, kernel_size=(3,3), activation='relu', padding='same')(l)
l = MaxPool2D(strides=(2,2))(c2)

c3 = Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same')(l)
l = MaxPool2D(strides=(2,2))(c3)

c4 = Conv2D(filters=32, kernel_size=(1,1), activation='relu', padding='same')(l)


l = concatenate([UpSampling2D(size=(2,2))(c4), c3], axis=-1)
l = Conv2D(filters=32, kernel_size=(2,2), activation='relu', padding='same')(l)
l = concatenate([UpSampling2D(size=(2,2))(l), c2], axis=-1)
l = Conv2D(filters=24, kernel_size=(2,2), activation='relu', padding='same')(l)
l = concatenate([UpSampling2D(size=(2,2))(l), c1], axis=-1)
l = Conv2D(filters=16, kernel_size=(2,2), activation='relu', padding='same')(l)
l = Conv2D(filters=64, kernel_size=(1,1), activation='relu')(l)
l = Dropout(0.5)(l)
output_layer = Conv2D(filters=3, kernel_size=(1,1), activation='sigmoid')(l)
                                                         
UNet = Model(input_layer, output_layer)

UNet.summary()

Compile U-Net model:

In [19]:
UNet.compile(optimizer=optimizer, loss=loss,metrics=['accuracy'])

Train U-Net model:

In [32]:
# Save best weights of model dependent on the lowest validation loss to load for testing
mcp = ModelCheckpoint(best_weights_filepath, monitor="val_loss",
                      save_best_only=True, save_weights_only=False)

# Train model
history =UNet.fit(x_data, y_data,
                epochs=epochs,
                batch_size=batch_size,
                shuffle=True,
                validation_split=validation_split,
                        callbacks = [mcp])

In [33]:
loss = history.history['loss']
val_loss = history.history['val_loss']

acc = history.history['acc']
val_acc = history.history['val_acc']

loss_fig = plt.figure()
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.plot(loss, 'blue', label='Training Loss')
plt.plot(val_loss, 'green', label='Validation Loss')
plt.xticks(range(0,epochs)[0::2])
plt.legend()
plt.show()
loss_fig.savefig('UNet_loss.png')


acc_fig = plt.figure()
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.plot(acc, 'blue', label='Training Accuracy')
plt.plot(val_acc, 'green', label='Validation Accuracy')
plt.xticks(range(0,epochs)[0::2])
plt.legend()
plt.show()
acc_fig.savefig('UNet_accuracy.png')

Load best weights and save final U-Net model:

In [25]:
# Load best weights form training
UNet.load_weights(best_weights_filepath)

# Save final model
filepath = "UNet.mod"
UNet.save(filepath)